Overview

In this part of the assessment, you will act as a Data analyst to answer a research question on the US elections. 

First, will be required to write a Data Report on the process that will undertake while working on the given research problem. Using the CRISP-DM Methodology,  you will document the various stages of the data science lifecycle for the given research problem while also providing your recommendation.

You have been provided with a detailed description of what you will be expected to do below.

Do remember that complex tasks that may seem hard at first can often be broken down into a sequence of simple tasks, and there are workarounds to do what first seems “impossible” with a succession of known operations.

Problem Description 

You have been asked to help a candidate to become US president. The winner of the election will be the candidate winning the most grand electors. 

Grand electors are attributed at the state level: in each of the 51 states, there is a given number of grand electors to win (roughly, but not exactly, proportional to the size of the state) and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.

Because the number of grand electors is not exactly proportional to the population, some states can be prioritized to increase the return on investment of the campaign. We assume here there are only 2 candidates, and no history (no trend of certain states to vote for a particular candidate or party). Hence, each vote is equally "expensive" to get, but some states grant more grand elector per capita.

You are provided with 2 tables: one giving the number of Grand Electors per state, the other the population per state.

You are asked to identify the states that should be prioritized to win the election, with a smart but simple algorithm (brute force computation of all possibilities to find the absolute optimum is not accepted, as it would be to computationally expensive). It is ok not to find the overall optimum, we just want a strategy that makes sense.

(This is of course a very simplistic view of reality, but it is a nice starting point to play with data and analyze possibilities).

First take a few minutes to think about what you need to do. Steps will be suggested hereafter, but a big part of the data scientist's job is to identify the flow of operations when being asked a practical question, so it is important you start exercising on that in addition to programming.

Here is what we are suggesting to do: we will rank states by decreasing number of grand electors per capita. The first states in the list will be the most valuable (you get a large number of grand electors by convincing a small number of people to vote for you). We will target all the states at the top of the list until the cumulative sum (also called running total) of grand electors won is larger than half the total number of Grand Electors in the country.

In [1]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

'Connected: @None'

In [2]:
# Importing the pandas library
# We will use a function read_csv from pandas to read our datasets as shown
#
import pandas as pd 
import csv

In [115]:
# Loading our table from the respective CSV files 
with open('GrandElectors_by_state.csv','r') as f:
    ELECTORS = pd.read_csv(f, index_col=0, encoding='utf-8')
# saving the urban center CSV file into a database
%sql DROP TABLE if EXISTS ELECTORS;
%sql PERSIST ELECTORS;

# Displaying the first  5 entries 
%sql SELECT * FROM ELECTORS LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [116]:
# Loading our table from the respective CSV files 
with open('Population_by_state.csv','r') as f:
    POPULATION_STATE = pd.read_csv(f, index_col=0, encoding='utf-8')
# saving the urban center CSV file into a database
%sql DROP TABLE if EXISTS POPULATION_STATE;
%sql PERSIST POPULATION_STATE;

# Displaying the first  5 entries 
%sql SELECT * FROM POPULATION_STATE LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


In [117]:
#Joining the tables
#First, convert the states name to Uppercase
%%sql
UPDATE ELECTORS
SET STATE = UPPER(STATE);
SELECT STATE, GRANDELECTORS FROM ELECTORS LIMIT 5


 * sqlite://
51 rows affected.
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55


In [79]:
%%sql
SELECT
    state,grandelectors,
    (SELECT Sum(grandelectors) FROM electors as r WHERE r.state <= t.state ) as "Cummulative Total"
FROM electors as t

Group By state;

 * sqlite://
Done.


State,GrandElectors,Cummulative Total
ALABAMA,9,9
ALASKA,3,12
ARIZONA,11,23
ARKANSAS,6,29
CALIFORNIA,55,84
COLORADO,9,93
CONNECTICUT,7,100
DC,3,103
DELAWARE,3,106
FLORIDA,29,135


In [120]:
#Joining the tables
%%sql
DROP TABLE if EXISTS ELECTION;
create table ELECTION as
SELECT * from POPULATION_STATE
join ELECTORS 
on POPULATION_STATE.STATE = ELECTORS.STATE

 * sqlite://
Done.
Done.


[]

In [128]:
#change the name of the "District of Columbia" state to its short version "DC"
%%sql
UPDATE ELECTION
SET STATE = "DC"
WHERE STATE ="DISTRICT OF COLUMBIA"

 * sqlite://
1 rows affected.


[]

In [134]:
#compute the ratio between the number of grand electors and the population. 
#Please create a new column with that ratio
%%sql
SELECT STATE,POPULATION,GRANDELECTORS, (POPULATION/GRANDELECTORS) AS RATIO
FROM ELECTION 

 * sqlite://
Done.


State,Population,GrandElectors,RATIO
CALIFORNIA,39144818,55,711723
TEXAS,27469114,38,722871
FLORIDA,20271272,29,699009
NEW YORK,19795791,29,682613
ILLINOIS,12859995,20,642999
PENNSYLVANIA,12802503,20,640125
OHIO,11613423,18,645190
GEORGIA,10214860,16,638428
NORTH CAROLINA,10042802,15,669520
MICHIGAN,9922576,16,620161


In [135]:
#order the states by decreasing ratio of Grand Electors per capita
%%sql
SELECT STATE,POPULATION,GRANDELECTORS, (POPULATION/GRANDELECTORS) AS RATIO
FROM ELECTION
ORDER BY RATIO ASC



 * sqlite://
Done.


State,Population,GrandElectors,RATIO
WYOMING,586107,3,195369
VERMONT,626042,3,208680
DC,672228,3,224076
ALASKA,738432,3,246144
NORTH DAKOTA,756927,3,252309
RHODE ISLAND,1056298,4,264074
SOUTH DAKOTA,858469,3,286156
DELAWARE,945934,3,315311
MAINE,1329328,4,332332
NEW HAMPSHIRE,1330608,4,332652


In [158]:
#compute the running total of Grand Electors
%%sql
SELECT
    STATE,POPULATION,GRANDELECTORS,(POPULATION/GRANDELECTORS) AS RATIO,(SELECT Sum(grandelectors) FROM ELECTION as r WHERE r.state <= t.state ) as "RUNNING TOTAL"
FROM ELECTION as t
GROUP BY STATE

 * sqlite://
Done.


State,Population,GrandElectors,RATIO,RUNNING TOTAL
ALABAMA,4858979,9,539886,9
ALASKA,738432,3,246144,12
ARIZONA,6828065,11,620733,23
ARKANSAS,2978204,6,496367,29
CALIFORNIA,39144818,55,711723,84
COLORADO,5456574,9,606286,93
CONNECTICUT,3590886,7,512983,100
DC,672228,3,224076,103
DELAWARE,945934,3,315311,106
FLORIDA,20271272,29,699009,135


In [137]:
#compute the half of the total of Grand Electors overall (in the whole country)
%%sql
SELECT SUM(GRANDELECTORS)/2 AS "HALF OF GRAND ELECTORS"
FROM ELECTION

 * sqlite://
Done.


HALF OF GRAND ELECTORS
269


In [160]:
#Select all the states for which the running total is below or equal to the threshold.
%%sql
SELECT
    STATE,POPULATION,GRANDELECTORS,
    (SELECT Sum(grandelectors) FROM electors as r WHERE r.state <= t.state ) as "RUNNING TOTAL"
FROM ELECTION as t
WHERE "RUNNING TOTAL" <=269
Group By state;

 * sqlite://
Done.


State,Population,GrandElectors,RUNNING TOTAL
ALABAMA,4858979,9,9
ALASKA,738432,3,12
ARIZONA,6828065,11,23
ARKANSAS,2978204,6,29
CALIFORNIA,39144818,55,84
COLORADO,5456574,9,93
CONNECTICUT,3590886,7,100
DC,672228,3,103
DELAWARE,945934,3,106
FLORIDA,20271272,29,135


In [161]:
#Add the first state for which the running total is larger than the threshold
%%sql
SELECT
    STATE,POPULATION,GRANDELECTORS,
    (SELECT Sum(grandelectors) FROM electors as r WHERE r.state <= t.state ) as "RUNNING TOTAL"
FROM ELECTION as t
WHERE "RUNNING TOTAL" <=275
Group By state;

 * sqlite://
Done.


State,Population,GrandElectors,RUNNING TOTAL
ALABAMA,4858979,9,9
ALASKA,738432,3,12
ARIZONA,6828065,11,23
ARKANSAS,2978204,6,29
CALIFORNIA,39144818,55,84
COLORADO,5456574,9,93
CONNECTICUT,3590886,7,100
DC,672228,3,103
DELAWARE,945934,3,106
FLORIDA,20271272,29,135
